### Resources
 
 'else' in a list comprehension<br>
 https://stackoverflow.com/questions/2951701/is-it-possible-to-use-else-in-a-list-comprehension
 
convert string to variable<br>
https://www.codeproject.com/Questions/1222606/Python-convert-string-and-variable-to-list-name

Apply function to each element of a list<br>
https://stackoverflow.com/questions/25082410/apply-function-to-each-element-of-a-list

In [5]:
# libraries, libraries!
import pandas as pd
import numpy as np # for 
import re # for RegEx
from re import search
from datetime import datetime

In [6]:
# set paths
pthPy         = r'C:\Users\hilton.netta\OneDrive - Prescient\py' # path to Py stuff in working folder
pthCompliance = r'P:\Investment Operations\GRC\Compliance' # path to PIM Compliance folder
pthW          = r'P:\Working Folders\Hilton\W'

In [74]:
# get regex structure dataframes
structures = pthPy + r'\structures.xlsx'

# import the regex date formats
dfrgx = pd.read_excel(structures, sheet_name       = 'dates')

# import the list of accrual line items, #https://datatofish.com/pandas-dataframe-to-series/
accr        = pd.read_excel(structures, sheet_name = 'accr', usecols = ['accruals'])
marg        = pd.read_excel(structures, sheet_name = 'accr', usecols = ['margins'])
accr_list   = [x for e in accr.values.tolist() for x in e] #https://bobbyhadz.com/blog/python-remove-square-brackets-from-list
margin_list = [x for e in marg.values.tolist() for x in e]

# import the regex CLN formats
clnrgx = pd.read_excel(structures, sheet_name      = 'cln')

# import the regex FRN formats
frnrgx = pd.read_excel(structures, sheet_name      = 'frn')

# import South African holidays
sa_hols = pd.read_excel(structures, sheet_name     = 'hols', usecols = ['sa_hols'])

# import issuer regex formats, sans NaNs # https://stackoverflow.com/questions/46218652/python-pandas-unique-value-ignoring-nan
issrgx = pd.read_excel(structures, sheet_name      = 'issuers', usecols = ['description', 'id', 'name']) # a pandas dataframe
#rgxid = issrgx['id'].dropna()

#issrgx_id = pd.read_excel(structures, sheet_name   = 'issuers', usecols = ['id', 'name'])
rgx_id = issrgx_id['id'].dropna() # this is now a pandas series

#issrgx_desc = pd.read_excel(structures, sheet_name = 'issuers', usecols = ['description', 'name'])
rgx_desc = issrgx_desc['description'].dropna() # this is now a pandas series

# import CLNs
clns = pd.read_excel(pthCompliance + r'\MCaps\CLNs.xlsx', sheet_name = 'CLN')

# import custodian accounts
sttlmnt = pd.read_excel(pthCompliance + r'\Daily\2A - Fund Codes, Breach Register.xlsx', sheet_name = 'Sttlmnt', usecols="A,K,L") # only "Fund", "Custodian", and "SAFEX" columns

# import CMS circular
med = pd.read_excel(pthCompliance + r'\Medical Schemes\20220117 MSA Circ6 Categorisations.xlsx', sheet_name = 'JSEEquity31Dec2021')

# import the Reg 28 file with all issuer codes assigned a named issuer
Reg28_basis = pd.read_excel(pthW + r'\!Reg28Worx 30Jun2022.xlsx')

In [76]:
# function to derive ISSUER from INSTRUMENT DESCRIPTION ("i Issue Name" column)
def issuer_desc(txt):
    for pattern in issrgx['description']:
        if re.search(pattern, txt.upper()):
            return issrgx.loc[issrgx['description'] == pattern].iat[0,2] # note 'break' within the for loop   
            break
        
test_txt_1 = 'The quick brown fox jumped mercedes over the lazy dog.'
print(issuer_desc(test_txt_1), isinstance(issuer_desc(test_txt_1), str)) # test if a variable is a string
# https://pythonprinciples.com/blog/check-if-var-is-string/

Mercedes-Benz South Africa (Pty) Ltd True


In [78]:
# function to derive ISSUER from INSTRUMENT DESCRIPTION ("i Issue Name" column)
def issuer_desc(txt):
    for pattern in rgx_desc:
        if re.search(pattern, txt.upper()):
            return issrgx.loc[issrgx['description'] == pattern].iat[0,2] # note 'break' within the for loop   
            break
        
test_txt_1 = 'The quick brown fox jumped mercedes over the lazy dog.'
print(issuer_desc(test_txt_1), isinstance(issuer_desc(test_txt_1), str)) # test if a variable is a string
# https://pythonprinciples.com/blog/check-if-var-is-string/

Mercedes-Benz South Africa (Pty) Ltd True


In [79]:
# function to derive ISSUER from INSTRUMENT ID ("Primary Asset ID" column)
def issuer_id(txt):
    for pattern in rgx_id:
        if re.search(pattern, txt.upper()): # check if the pattern exists in the txt
            return issrgx.loc[issrgx['id'] == pattern].iat[0,2] # note the 'break' within the for loop 
            break

test_txt_2 = 'SBT054 the lazy dog'
print(issuer_id(test_txt_2))

Standard Bank Group Ltd


In [80]:
# utility function to open an excel file, .xls or .xlsx
def open_xl_file(file_name_and_path):
    import win32com.client as win32 # library to convert xls to xlsx
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.DisplayAlerts = False # suppress the warning dialogue
    excel.Workbooks.Open(file_name_and_path)
    excel.DisplayAlerts = True # unsuppress the warning dialogue

In [81]:
# function that extracts date from string
def datex(txt):
    try:
        for pattern in dfrgx['date_regex']:
            if re.search(pattern, txt.title()):
                break
        return datetime.strptime(re.search(pattern, txt.title()).group(),dfrgx.loc[dfrgx['date_regex'] == pattern].iat[0,1]).strftime("%d%b%Y")

    # manage exceptions:
    except ValueError as ve:
        print(f'ValueError {ve}')
    except TypeError as te:
        print(f'TypeError {te}')
    except AttributeError as ae:
        print(f'AttributeError {ae}')
        
#test the datex function
txt = "BNP Paribas Personal Finance SA Ltd FRN BPPF31 Jb3+95 17-Aug-13"
datex(txt)

'17Aug2013'

In [88]:
# functions to extract CLN, FRN, NCD, ILB
def cln(txt):
    for pattern in clnrgx['regex']:
        if re.search(pattern, txt.upper()): 
            #return clnrgx.loc[clnrgx['regex'] == pattern].iat[0,2] # note 'break' below the first 'for'   
            return 1
        break
        
def frn(txt):
    for pattern in frnrgx['regex']:
        if re.search(pattern, txt.upper()): 
            #return frnrgx.loc[frnrgx['regex'] == pattern].iat[0,2] # note 'break' below the first 'for' 
            return 1
        break
        
# import the regex ILB formats
ilbrgx = pd.read_excel(structures, sheet_name = 'ilb')
def ilb(txt):
    for pattern in ilbrgx['regex']:
        if re.search(pattern, txt.upper()): 
            #return ilbrgx.loc[ilbrgx['regex'] == pattern].iat[0,2] # note the 'break' below the first 'for'  
            return 1
        break       
        
txt = 'de Barge inflation asn'
print(frn(txt), type(frn(txt)), cln(txt), type(cln(txt)), ilb(txt), type(ilb(txt)))

None <class 'NoneType'> 1 <class 'int'> 1 <class 'int'>


In [89]:
# function that creates a new column based on the values of othe columns
# Check if an element is in a list https://www.geeksforgeeks.org/check-if-element-exists-in-list-in-python/
def categorise(row):
    t = issuer_desc(row['i Issue Name'])                                   # create a temp so the function only gets called once
    g = issuer_id(row['Primary Asset ID'])                                 # create a temp so the function only gets called once
    if row['Primary Asset ID'] in accr_list:                               # item is an accrual
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,1]
    elif row['Investment Type'] == 'SYTH' and row['CCY'] != 'ZAR':         # item is non-ZAR synthetic cash
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,1]
    elif row['Investment Type'] == 'SYTH' and row['CCY'] == 'ZAR':         # item is ZAR synthetic cash
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,2]
    elif row['Primary Asset ID'] in margin_list:                           # item is a margin account (initial or variation)
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,2]
    #elif cln(row['i Issue Name']) == 1:                                    # item is a CLN
        #return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,2]
    elif isinstance(t, str):                                               # INSTRUMENT DESCRIPTION is a hit ...
        return t
    elif isinstance(g, str):                                               # INSTRUMENT ID is a hit ...
        return g
    return '-xxx-'

In [90]:
# (1) read in a Reg 28 file from Eagle as a dataframe called 'example' to use as an input
example = pd.read_excel(pthW + r'\Reg 28 Report - test.xlsx')

In [91]:
# (2) identify instrument attributes
# Map an if statement in Python: https://stackoverflow.com/questions/29247718/map-an-if-statement-in-python
example['CLN']    = example['i Issue Name'].map(cln)
example['FRN']    = example['i Issue Name'].map(frn)
example['ILB']    = example['i Issue Name'].map(ilb)
example['Date']   = example['i Issue Name'].map(datex)
example['Issuer'] = example.apply(lambda row: categorise(row), axis=1)

AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError

In [92]:
# (3) identify absent issuers
no_issuer = example[example['Issuer'] == '-xxx-']
print(len(no_issuer.Issuer))

0


In [93]:
# (4) write the results to a new excel file
writer = pd.ExcelWriter(pthW + r'\Reg 28 Report - results.xlsx', engine = 'xlsxwriter')

example.to_excel(   writer, index = False, sheet_name = 'results')
no_issuer.to_excel( writer, index = False, sheet_name = 'no issuers')

writer.save()
#open_xl_file(      pthW + r'\Reg 28 Report - result2.xlsx') # open the results file

'C:\\Users\\hilton.netta\\OneDrive - Prescient\\py\\structures.xlsx'

In [10]:
# delete a column in the 'example' dataframe
example = example.drop('Issuer_Name', axis=1)
example = example.drop('CLN', axis=1) 
example = example.drop('FRN', axis=1) 
example = example.drop('ILB', axis=1) 
example = example.drop('Date', axis=1)

NameError: name 'example' is not defined